In [1]:
import os
if os.path.exists("/gss_gpfs_scratch/"):
    RMG_PY_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-Py/')
    RMG_MODELS_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-models')
else:
    RMG_PY_PATH = os.path.expanduser('~/Code/RMG-discovery/')
    RMG_MODELS_PATH = os.path.expanduser('~/Code/RMG-models')



import IPython
from IPython.display import display
import sys
import re

sys.path.insert(1,RMG_PY_PATH) # a copy of RMG-Py on the `importer` branch.
from rmgpy.molecule import Molecule
import rmgpy.kinetics
import numpy
import cPickle as pickle
from collections import Counter, defaultdict
from rmgpy.reaction import Reaction
from rmgpy import reaction

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import rmgpy.rmg
import rmgpy.data
import rmgpy.data.kinetics
import ck2cti
import numpy as np
import matplotlib.pyplot as plt

In [2]:
h = open("../reference_files/importerKinetics.pkl", "r")
importer_kinetics = pickle.load(h)
importer_kinetics = pd.DataFrame(importer_kinetics)
importer_kinetics = importer_kinetics.drop("AutoTST-OOHabstraction")
importer_kinetics

,CHCHCH3 + IC8H18 => C3H6 + IC8H17,CH2CCH3 + IC8H18 => C3H6 + IC8H17,CH2C3H5 + IC8H18 => NC4H8 + IC8H17,CH2OH + IC8H18 => CH3OH + IC8H17,DC5H11 + CO <=> IC5H11CO,HCCO + IC8H18 => CH2CO + IC8H17,C4H8CHO-1 + H2O <=> NC4H9CHO + OH,CH3CO + IC8H18 => CH3CHO + IC8H17,mf25oh + h <=> f2oh + ch3,mf25oh + h <=> mf2 + oh,...,C11OOH1-3O2 <=> C11OOH1-3 + O2,OHX + O2 <=> HO2 + O,C11OOH1-4O2 <=> C11OOH1-4 + O2,IC3H7OO + CYC6H8 => C3H7OOH + C6H6 + H,A2- + C4H4 <=> A3 + H,C11OOH2-1O2 <=> C11OOH2-1 + O2,CH3TXcC6H10 + O2 <=> CH3-1-cC6H9 + HO2,C11OOH3-2O2 <=> C11OOH3-2 + O2,SC4H7 + NC3H7OH => NC4H8 + CH3CHCH2OH,sc4h9o2 + c4h10 <=> sc4h9o2h + sc4h9
AramcoMech_1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(1.12e+13,'cm^3/(mol*s)'), n=0, Ea..."
AramcoMech_2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Biomass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chernov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2012/2028-Sarathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(1.12e+13,'cm^3/(mol*s)'), n=0, Ea..."
CombFlame2013/1315-Chang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1541-Zhang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(1.12e+13,'cm^3/(mol*s)'), n=0, Ea..."
CombFlame2013/1609-Veloo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/17-Malewicki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(1.12e+13,'cm^3/(mol*s)'), n=0, Ea..."
CombFlame2013/1907-Merchant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
f = open("../../autotst_kinetics.pkl","r")
autotst_kinetics = pickle.load(f)
ooh_reactions = []
for rxn in autotst_kinetics:
    reactants, products = rxn.label.split("_")
    r1, r2 = reactants.split('+')
    p1, p2 = products.split('+')
    ooh_smiles = [r1, r2, p1, p2]
    if "OO" in ooh_smiles and "[O]O" in ooh_smiles:
        
        inchikey_to_smiles_dict = {}
        for smiles in ooh_smiles:
            inchikey_to_smiles_dict[Molecule(SMILES=smiles).toInChIKey()] = smiles
        
    
        for reactant in rxn.reactants:
            inchi_key = reactant.label.split("-u")[0]
            reactant.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
            reactant.label = inchikey_to_smiles_dict[inchi_key]


        for product in rxn.products:
            inchi_key = product.label.split("-u")[0]
            product.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
            product.label = inchikey_to_smiles_dict[inchi_key]
        
        #print [r1, r2, p1, p2]
        ooh_reactions.append(rxn)
        

In [4]:
def RateCoefficients(arrhenius):
    """
    Takes an Arrhenius object, evaluates k at 1000K 1bar, and returns log10(k)
    """
    rate = np.nan
    try:
        rate = np.log10(arrhenius.getRateCoefficient(T=1000, P=1e5))
        print "Rate Calculated"
    except:
        print "No rate to calculate"
        pass
    return rate

In [5]:
combined_kinetics = pd.DataFrame(index=np.append(importer_kinetics.index.values, "AutoTST"))
for ooh_reaction in ooh_reactions:
    single_rxn_kinetics = importer_kinetics.T[importer_kinetics.columns.map(ooh_reaction.isIsomorphic)].T
    if single_rxn_kinetics.shape[-1] == 2:
        rxn1, rxn2 =  single_rxn_kinetics.columns
        
        if reaction._isomorphicSpeciesList(rxn1.reactants, ooh_reaction.reactants):
            single_rxn_kinetics = single_rxn_kinetics[rxn1]
        else:
            single_rxn_kinetics = single_rxn_kinetics[rxn2]
    elif single_rxn_kinetics.shape[-1] == 0:
        continue
    #single_rxn_kinetics.loc["AutoTST"] = ooh_reaction.kinetics
    single_rxn_kinetics.loc["AutoTST"] = ooh_reaction.kinetics
    combined_kinetics[ooh_reaction] = single_rxn_kinetics 
    print ooh_reaction.kinetics
    
#single_rxn_kinetics.dropna().map(RateCoefficients)

Arrhenius(A=(2.8487e-06,'cm^3/(mol*s)'), n=4.92965, Ea=(44.8359,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.63671, dn = +|- 0.0646606, dEa = +|- 0.355698 kJ/mol""")
Arrhenius(A=(0.00319676,'cm^3/(mol*s)'), n=4.00953, Ea=(53.9129,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.09715, dn = +|- 0.0121683, dEa = +|- 0.0669379 kJ/mol""")
Arrhenius(A=(4.28897e-08,'cm^3/(mol*s)'), n=5.38606, Ea=(14.9036,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.97671, dn = +|- 0.0894318, dEa = +|- 0.491964 kJ/mol""")
Arrhenius(A=(8.26892e-07,'cm^3/(mol*s)'), n=5.28799, Ea=(63.0482,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 2.14314, dn = +|- 0.100041, dEa = +|- 0.550328 kJ/mol""")
Arrhenius(A=(7.99218e-05,'cm^3/(mol*s)'), n=4.66681, Ea=(61.9057,'kJ/mol'), T0=(1,'K')

In [6]:
combined_kinetics

,[O]O + CC(=O)C=C(C)C <=> OO + [CH2]C(=O)C=C(C)C,CC(=O)C(C)(C)C + [O]O <=> OO + [CH2]C(C)(C)C(C)=O,OO + CCCCCC(C)O[O] <=> CCCCCC(C)OO + [O]O,CC1CCCCC1 + [O]O <=> OO + CC1C[CH]CCC1,CCC(C)O + [O]O <=> OO + [CH2]CC(C)O,CCCC(C)C=O + [O]O <=> OO + CCC[C](C)C=O,CC(C)CC(C)O[O] + OO <=> CC(C)CC(C)OO + [O]O,[O]O + CC=CCC <=> OO + C[CH]C=CC,CCO + [O]O <=> [CH2]CO + OO,C=CCC=C + [O]O <=> C=C[CH]C=C + OO,...,O=C=COC=O + [O]O <=> O=[C]OC=C=O + OO,OO + CC(O)CCO[O] <=> [O]O + CC(O)CCOO,CCCC=O + [O]O <=> C[CH]CC=O + OO,CC(C)CC(C)CO[O] + OO <=> CC(C)CC(C)COO + [O]O,[O]O + COC(=O)CCCOO <=> COC(=O)CCCO[O] + OO,[O]O + C=C(C)C(=O)CC <=> C=C(C)C(=O)[CH]C + OO,[O]O + CCCCCCCCCCC <=> OO + CC[CH]CCCCCCCC,[O]O + C=CCCCC <=> OO + C=CC[CH]CC,OO + CC(C)CCC(C)O[O] <=> CC(C)CCC(C)OO + [O]O,CCCC(C)=O + [O]O <=> OO + [CH2]C(=O)CCC
AramcoMech_1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(0.03986,'cm^3/(mol*s)'), n=4.3, E...",NaN,...,NaN,NaN,"Arrhenius(A=(9640,'cm^3/(mol*s)'), n=2.6, Ea=(...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
AramcoMech_2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(4820,'cm^3/(mol*s)'), n=2.55, Ea=...","Arrhenius(A=(0.03986,'cm^3/(mol*s)'), n=4.3, E...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Biomass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chernov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(24300,'cm^3/(mol*s)'), n=2.5, Ea=...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2012/2028-Sarathy,NaN,NaN,NaN,NaN,"Arrhenius(A=(0.000414,'cm^3/(mol*s)'), n=4.76,...",NaN,NaN,NaN,"Arrhenius(A=(0.00904,'cm^3/(mol*s)'), n=4.54, ...",NaN,...,NaN,"Arrhenius(A=(2.4e+12,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(9640,'cm^3/(mol*s)'), n=2.6, Ea=(...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1315-Chang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1541-Zhang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(23790,'cm^3/(mol*s)'), n=2.55, Ea...",NaN,...,NaN,NaN,"Arrhenius(A=(58800,'cm^3/(mol*s)'), n=2.5, Ea=...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1609-Veloo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/17-Malewicki,NaN,NaN,"Arrhenius(A=(2.4e+12,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,NaN,NaN,"Arrhenius(A=(4820,'cm^3/(mol*s)'), n=2.55, Ea=...","Arrhenius(A=(12300,'cm^3/(mol*s)'), n=2.55, Ea...",NaN,...,NaN,NaN,"Arrhenius(A=(9640,'cm^3/(mol*s)'), n=2.6, Ea=(...",NaN,NaN,NaN,"Arrhenius(A=(1.12e+13,'cm^3/(mol*s)'), n=0, Ea...",NaN,NaN,"Arrhenius(A=(23800,'cm^3/(mol*s)'), n=2.55, Ea..."
CombFlame2013/1907-Merchant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from copy import deepcopy

rate_kin = combined_kinetics.applymap(RateCoefficients)
rate_kin

No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
Rate Calculated
Rate Calculated
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
Rate Calculated
Rate Calculated
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
No rate to calculate
Rate Calculated
No rate to calculate
No rate to calcul

,[O]O + CC(=O)C=C(C)C <=> OO + [CH2]C(=O)C=C(C)C,CC(=O)C(C)(C)C + [O]O <=> OO + [CH2]C(C)(C)C(C)=O,OO + CCCCCC(C)O[O] <=> CCCCCC(C)OO + [O]O,CC1CCCCC1 + [O]O <=> OO + CC1C[CH]CCC1,CCC(C)O + [O]O <=> OO + [CH2]CC(C)O,CCCC(C)C=O + [O]O <=> OO + CCC[C](C)C=O,CC(C)CC(C)O[O] + OO <=> CC(C)CC(C)OO + [O]O,[O]O + CC=CCC <=> OO + C[CH]C=CC,CCO + [O]O <=> [CH2]CO + OO,C=CCC=C + [O]O <=> C=C[CH]C=C + OO,...,O=C=COC=O + [O]O <=> O=[C]OC=C=O + OO,OO + CC(O)CCO[O] <=> [O]O + CC(O)CCOO,CCCC=O + [O]O <=> C[CH]CC=O + OO,CC(C)CC(C)CO[O] + OO <=> CC(C)CC(C)COO + [O]O,[O]O + COC(=O)CCCOO <=> COC(=O)CCCO[O] + OO,[O]O + C=C(C)C(=O)CC <=> C=C(C)C(=O)[CH]C + OO,[O]O + CCCCCCCCCCC <=> OO + CC[CH]CCCCCCCC,[O]O + C=CCCCC <=> OO + C=CC[CH]CC,OO + CC(C)CCC(C)O[O] <=> CC(C)CCC(C)OO + [O]O,CCCC(C)=O + [O]O <=> OO + [CH2]C(=O)CCC
AramcoMech_1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.149583,NaN,...,NaN,NaN,2.744113,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AramcoMech_2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.031766,2.149583,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Biomass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chernov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.432231,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2012/2028-Sarathy,NaN,NaN,NaN,NaN,1.607895,NaN,NaN,NaN,2.960641,NaN,...,NaN,4.194759,2.744113,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1315-Chang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1541-Zhang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.422584,NaN,...,NaN,NaN,3.021795,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1609-Veloo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/17-Malewicki,NaN,NaN,4.194759,NaN,NaN,NaN,NaN,3.031766,2.297818,NaN,...,NaN,NaN,2.744113,NaN,NaN,NaN,3.183153,NaN,NaN,2.816148
CombFlame2013/1907-Merchant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
rate_kin[rate_kin.subtract(rate_kin.loc["AutoTST"]).abs().count(axis=1).sort_values() > 50].subtract(rate_kin.loc["AutoTST"]).abs().mean(axis=1).sort_values()


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


AutoTST                       0.000000
PCI2013/289-Dagaut            0.841436
MB-Fisher                     1.205897
MB-Farooq                     1.210888
CombFlame2012/2028-Sarathy    1.342094
PCI2013/411-Darcy             1.389910
CombFlame2013/17-Malewicki    1.427612
PCI2017/036-Rashidi           1.432653
PCI2013/353-Malewicki         1.446481
n-Heptane                     1.468690
Gasoline_2                    1.471617
Gasoline_Surrogate            1.496109
PCI2013/361-Malewicki         1.534629
PCI2013/335-Wang              1.541382
dtype: float64

In [9]:
rate_kin[rate_kin.subtract(rate_kin.loc["AutoTST"]).abs().count(axis=1).sort_values() > 50].subtract(rate_kin.loc["AutoTST"]).abs().count(axis=1).sort_values()


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


CombFlame2012/2028-Sarathy     58
PCI2013/289-Dagaut             64
MB-Fisher                      67
MB-Farooq                      69
PCI2013/353-Malewicki          77
n-Heptane                      91
PCI2013/411-Darcy             102
CombFlame2013/17-Malewicki    108
PCI2013/335-Wang              124
Gasoline_Surrogate            148
Gasoline_2                    163
PCI2013/361-Malewicki         164
PCI2017/036-Rashidi           179
AutoTST                       400
dtype: int64

Choosing to look at the PCI2013/335-Wang model for multiple reasons:

- Large number of reactions calculated for this model
- Highest disagreement with AutoTST calculations
- Experimental data readily available
- Model appears to have disagreements with experimenal data

## This portion of the code is designed to write `.yaml` files baised on the experimental data

In [10]:
equi = [
    [0.5, 0.0122, 0.2075, 0.7803],
    [1.0, 0.0241, 0.2050, 0.7709],
    [2.0, 0.0471, 0.2002, 0.7527]
]

equi = pd.DataFrame(equi)
equi.columns = ["Phi", "X", "O2", "N2"]
#equi.set_index("Phi")
equi

,Phi,X,O2,N2
0,0.5,0.0122,0.2075,0.7803
1,1.0,0.0241,0.2050,0.7709
2,2.0,0.0471,0.2002,0.7527


In [12]:
xls = pd.ExcelFile("../reference_files/wang_data/wang_shock_tube.xlsx")
NUIG_df = pd.read_excel(xls, "NUIG shock tube")
RPI_df = pd.read_excel(xls, "RPI shock tube")
NUIG_df.columns = ["Phi", "T", "P", "Tao"]
RPI_df.columns = ["Phi", "T", "P", "Tao"]
#NUIG_df.set_index("Phi")
#RPI_df.set_index("Phi")

In [13]:
NUIG_df

,Phi,T,P,Tao
0,0.5,1086,6.66,984
1,0.5,1089,7.11,1072
2,0.5,1169,7.00,560
3,0.5,1253,6.56,151
4,0.5,1261,6.04,116
5,0.5,1058,10.79,1625
6,0.5,1097,9.47,700
7,0.5,1211,9.38,192
8,0.5,1311,8.24,54
9,1.0,1091,7.34,1176


In [14]:

RPI_init_text = """---
file-author:
    name: Nathan Harms
    ORCID: 0000-0003-2680-371X
file-version: 0
chemked-version: 0.0.1
reference:
    doi: 10.1016/j.proci.2012.06.001
    authors:
        - name: Weijing Wang et al.
    journal: Proceedings of the Combustion Institute
    year: 2013
    volume: 34
    pages: 335-343
    detail: m-h_{1}_xO2={2}_phi={0}
experiment-type: ignition delay
apparatus:
    kind: shock tube
    institution: RPI
    facility: stainless steel shock tube
common-properties:
    composition: &comp
      kind: mole fraction
      species:
        - species-name: 3-methyl-heptane
          InChI: 1S/C8H18/c1-4-6-7-8(3)5-2/h8H,4-7H2,1-3H3
          amount:
            - {1} 
        - species-name: O2
          InChI:  1S/O2/c1-2
          amount:
            - {2} 
        - species-name: N2
          InChI: 1S/N2/c1-2
          amount:
            - {3} 
    ignition-type:  &ign 
        target: OH*
        type: d/dt max
datapoints:"""


NUIG_init_text = """---
file-author:
    name: Nathan Harms
    ORCID: 0000-0003-2680-371X
file-version: 0
chemked-version: 0.0.1
reference:
    doi: 10.1016/j.proci.2012.06.001
    authors:
        - name: Weijing Wang et al.
    journal: Proceedings of the Combustion Institute
    year: 2013
    volume: 34
    pages: 335-343
    detail: m-h_{1}_xO2={2}_phi={0}
experiment-type: ignition delay
apparatus:
    kind: shock tube
    institution: NUIG
    facility: stainless steel shock tube
common-properties:
    composition: &comp
      kind: mole fraction
      species:
        - species-name: 3-methyl-heptane
          InChI: 1S/C8H18/c1-4-6-7-8(3)5-2/h8H,4-7H2,1-3H3
          amount:
            - {1} 
        - species-name: O2
          InChI:  1S/O2/c1-2
          amount:
            - {2} 
        - species-name: N2
          InChI: 1S/N2/c1-2
          amount:
            - {3} 
    ignition-type:  &ign 
        target: OH*
        type: d/dt max
datapoints:"""

In [15]:
data_point = """
    - temperature:
        - {0} kelvin
      ignition-delay:
        - {1} us
      pressure:
        - {2} atm
      composition: *comp
      ignition-type: *ign
      equivalence-ratio: {3}"""

In [18]:
for en in equi.values:
    pphi, X, O2, N2 = en
    file_name = "Wang_phi_{0}_m-h_{1}_xO2_{2}_NUIG.yaml".format(pphi, X, O2)
    write_string = NUIG_init_text.format(pphi, X, O2, N2)
    for entry in NUIG_df[NUIG_df.Phi == pphi].values:
        phi, P, T, tau = entry
        write_string += data_point.format(T, tau, P, phi) 
    print write_string
    f = open("../reference_files/wang_data/" + file_name, "w")
    f.write(write_string)
    print

---
file-author:
    name: Nathan Harms
    ORCID: 0000-0003-2680-371X
file-version: 0
chemked-version: 0.0.1
reference:
    doi: 10.1016/j.proci.2012.06.001
    authors:
        - name: Weijing Wang et al.
    journal: Proceedings of the Combustion Institute
    year: 2013
    volume: 34
    pages: 335-343
    detail: m-h_0.0122_xO2=0.2075_phi=0.5
experiment-type: ignition delay
apparatus:
    kind: shock tube
    institution: NUIG
    facility: stainless steel shock tube
common-properties:
    composition: &comp
      kind: mole fraction
      species:
        - species-name: 3-methyl-heptane
          InChI: 1S/C8H18/c1-4-6-7-8(3)5-2/h8H,4-7H2,1-3H3
          amount:
            - 0.0122 
        - species-name: O2
          InChI:  1S/O2/c1-2
          amount:
            - 0.2075 
        - species-name: N2
          InChI: 1S/N2/c1-2
          amount:
            - 0.7803 
    ignition-type:  &ign 
        target: OH*
        type: d/dt max
datapoints:
    - temperature:
        

In [19]:
for en in equi.values:
    pphi, X, O2, N2 = en
    file_name = "Wang_phi_{0}_m-h_{1}_xO2_{2}_RPI.yaml".format(pphi, X, O2)
    write_string = RPI_init_text.format(pphi, X, O2, N2)
    for entry in RPI_df[RPI_df.Phi == pphi].values:
        phi, P, T, tau = entry
        write_string += data_point.format(T, tau, P, phi) 
    print write_string 
    f = open("../reference_files/wang_data/" + file_name, "w")
    f.write(write_string)
    print

---
file-author:
    name: Nathan Harms
    ORCID: 0000-0003-2680-371X
file-version: 0
chemked-version: 0.0.1
reference:
    doi: 10.1016/j.proci.2012.06.001
    authors:
        - name: Weijing Wang et al.
    journal: Proceedings of the Combustion Institute
    year: 2013
    volume: 34
    pages: 335-343
    detail: m-h_0.0122_xO2=0.2075_phi=0.5
experiment-type: ignition delay
apparatus:
    kind: shock tube
    institution: RPI
    facility: stainless steel shock tube
common-properties:
    composition: &comp
      kind: mole fraction
      species:
        - species-name: 3-methyl-heptane
          InChI: 1S/C8H18/c1-4-6-7-8(3)5-2/h8H,4-7H2,1-3H3
          amount:
            - 0.0122 
        - species-name: O2
          InChI:  1S/O2/c1-2
          amount:
            - 0.2075 
        - species-name: N2
          InChI: 1S/N2/c1-2
          amount:
            - 0.7803 
    ignition-type:  &ign 
        target: OH*
        type: d/dt max
datapoints:
    - temperature:
        -